## Models

In [12]:
from typing import List

class Rectangle:
    
    def __init__(self, width=0, height=0, x=0, y=0, ident=1):
        self.width = width
        self.height = height
        self.x = x
        self.y = y
        self.ident = ident
    
    def __str__(self):
        return f'{self.ident}: ({self.width},{self.height},{self.x},{self.y}) whxy'
    
def decodeIntList(lst: List[str]) -> List[int]:
    return list(map(int, lst))

def decodeFloatList(lst: List[str]) -> List[float]:
    return list(map(float, lst))

def decodeRectangles(lst: List[str]) -> List[Rectangle]:
    return list(map(decodeRectangle, lst))

def decodeRectangle(rec: str) -> Rectangle:
    x,y,w,h = rec.split(',')
    return Rectangle(x=int(x), y=int(y), width=int(w), height=int(h))


class Individual:
    def __init__(self, iteration: int, objectiveValue: int, facilitySequence: List[int], facilitySequenceDecoded: List[int], facilitySequenceRandomKey: List[float], slicingOrderDecoded: List[int], slicingOrderRandomKey: List[float], orientations: List[str], facilityPlacement: List[Rectangle]):
        self.iteration = iteration
        self.objectiveValue = objectiveValue
        self.facilitySequence = facilitySequence
        self.facilitySequenceDecoded = facilitySequenceDecoded
        self.facilitySequenceRandomKey = facilitySequenceRandomKey
        self.slicingOrderDecoded = slicingOrderDecoded
        self.slicingOrderRandomKey = slicingOrderRandomKey
        self.orientations = orientations
        self.facilityPlacement = facilityPlacement
        
    @staticmethod
    def create(json: dict) -> 'Individual':
        return Individual(
            iteration=int(json['iteration']),
            objectiveValue=float(json['objectiveValue']),
            facilitySequence=decodeIntList(json['facilitySequence']),
            facilitySequenceDecoded=decodeIntList(json['facilitySequenceDecoded']),
            facilitySequenceRandomKey=decodeFloatList(json['facilitySequenceRandomKey']),
            slicingOrderDecoded=decodeIntList(json['slicingOrderDecoded']),
            slicingOrderRandomKey=decodeFloatList(json['slicingOrderRandomKey']),
            orientations=json['orientations'],
            facilityPlacement=decodeRectangles(json['facilityPlacement_XYWH']))

class HallOfFameRecord:
    def __init__(self, iteration: int, objectiveMin: float, objectiveMax: float, objectiveAvg: float, bestIndividual: Individual):
        self.iteration = iteration
        self.objectiveMin = objectiveMin
        self.objectiveMax = objectiveMax
        self.objectiveAvg = objectiveAvg
        self.bestIndividual = bestIndividual
    
    @staticmethod
    def create(json: dict) -> 'HallOfFameRecord':
        return HallOfFameRecord(
            iteration=int(json['iteration']),
            objectiveMin=float(json['objectiveMin']),
            objectiveMax=float(json['objectiveMax']),
            objectiveAvg=float(json['objectiveAvg']),
            bestIndividual=Individual.create(json['bestIndividual']))
            
class GaResult:
    def __init__(self, bestIndividual: Individual, hallOfFame: List[HallOfFameRecord]):
        self.bestIndividual = bestIndividual
        self.hallOfFame = hallOfFame
        
    @staticmethod
    def create(json: dict) -> 'GaResult':
        return GaResult(
            bestIndividual=Individual.create(json['bestIndividual']),
            hallOfFame=list(map(HallOfFameRecord.create, json['hallOfFame']['records'])))
    
class GaParameters:
    def __init__(self, mutationProb: float, crossoverProb: float, maxNumberOfIter: int, populationSize: int, geneticAlgorithm: str, mate: str, mutate: str, select: str, metric: str, objective: str):
        self.mutationProb = mutationProb
        self.crossoverProb = crossoverProb
        self.maxNumberOfIter = maxNumberOfIter
        self.populationSize = populationSize
        self.geneticAlgorithm = geneticAlgorithm
        self.mate = mate
        self.mutate = mutate
        self.select = select
        self.metric = metric
        self.objective = objective
        
    @staticmethod
    def create(json: dict) -> 'GaParameters':
        return GaParameters(
            mutationProb=float(json['mutationProb']),
            crossoverProb=float(json['crossoverProb']),
            maxNumberOfIter=int(json['maxNumberOfIter']),
            populationSize=int(json['populationSize']),
            geneticAlgorithm=json['geneticAlgorithm'],
            mate=json['mate'],
            mutate=json['mutate'],
            select=json['select'],
            metric=json['metric'],
            objective=json['objective'])

class Layout:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        
    @staticmethod
    def create(json: dict) -> 'Layout':
        return Layout(width=json['width'], height=json['height'])
    

class Dimension:
    def __init__(self, facility: str, area: float):
        self.facility = facility
        self.area = area
        
    @staticmethod
    def create(json: dict) -> 'Dimension':
        return Dimension(facility=json['facility'], area=float(json['area']))
        
class Flow:
    def __init__(self, ffrom: str, to: str, flow: float):
        self.ffrom = ffrom
        self.to = to
        self.flow = flow
        
    @staticmethod
    def create(json: dict) -> 'Flow':
        return Flow(ffrom=json['from'], to=json['to'], flow=float(json['flow']))
    
class InstanceParameters:
    def __init__(self, facilityCount: int, emptySpace: int, layout: Layout, dimension: List[Dimension], flow: List[Flow]):
        self.facilityCount = facilityCount
        self.emptySpace = emptySpace
        self.layout = layout
        self.dimension = dimension
        self.flow = flow
        
    @staticmethod
    def create(json: dict) -> 'InstanceParameters':
        return InstanceParameters(
            facilityCount=int(json['facilityCount']),
            emptySpace=int(json['emptySpace']),
            layout=Layout.create(json['layout']),
            dimension=list(map(Dimension.create, json['dimension'])),
            flow=list(map(Flow.create, json['flow'])))
    
    
class ComputationResult:
    def __init__(self, durationMillis: int, gaResult: GaResult, gaParameters: GaParameters, instanceParameters: InstanceParameters):
        self.durationMillis = durationMillis
        self.gaResult = gaResult
        self.gaParameters = gaParameters
        self.instanceParameters = instanceParameters
    
    @staticmethod
    def createWithName(name: str, json: dict) -> 'ComputationResult':
        res = ComputationResult.create(json)
        res.name = name
        return res
        
    @staticmethod
    def create(json: dict) -> 'ComputationResult':
        return ComputationResult(
            durationMillis=int(json['durationMillis']),
            gaResult=GaResult.create(json['gaResult']),
            gaParameters=GaParameters.create(json['gaParameters']),
            instanceParameters=InstanceParameters.create(json['instanceParameters']))

In [2]:
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 150

def draw_rectangles(recs: List[Rectangle], grid: Rectangle, show_ident=True) -> None:
    def to_matplotlib_rec(rec):
        return matplotlib.patches.Rectangle(
            (rec.x, rec.y),
            width=rec.width,height=rec.height,
            edgecolor='blue',
            linewidth=1.3,
            fill=False,
        )
    fig, ax = plt.subplots()
    
    for rec in recs:
        ax.add_patch(to_matplotlib_rec(rec))
        if show_ident:
            ax.text(rec.x+rec.width/2, rec.y+rec.height/2, rec.ident, fontsize=8)


    ax.set_xlim(left=-1, right=grid.width+1)
    ax.set_ylim(bottom=-1, top=grid.height)
    ax.set_title(f'Slicing layout', fontsize=8)
    
    # Hide the right and top spines
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)    
    
    plt.show()

## Loading computation result

In [17]:
from pathlib import Path
import json

DATASET_DIR = '/home/slarty/Documents/master-thesis/src/impl/out'
COMPUTATIONAL_RESULT_FILE_NAME = 'computation_result.json'

def _getDirPrefix(name: str) -> int:
    return int(name.split('_')[0])

def loadLatestComputationResult() -> ComputationResult:
    dirs = [x for x in Path(DATASET_DIR).iterdir() if x.is_dir()]
    latest, prefix = dirs[0], _getDirPrefix(dirs[0].name)
    for d in dirs:
        k = _getDirPrefix(d.name)
        if k > prefix:
            latest, prefix = d, k
    return loadComputationResult(str(latest.resolve()))
    

def loadComputationResult(path: str) -> ComputationResult:
    print(f'loading: {path}')
    with open(path + '/' + COMPUTATIONAL_RESULT_FILE_NAME, 'r') as f:
        data = json.load(f)
    return ComputationResult.createWithName(path.split('/')[-1], data)
    
df = loadLatestComputationResult()
df.name

loading: /home/slarty/Documents/master-thesis/src/impl/out/005_simpleGa_vC10_8555E


'005_simpleGa_vC10_8555E'

## Drawing

In [4]:
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 150

def draw_rectangles(recs: List[Rectangle], grid: Rectangle, show_ident=True) -> None:
    def to_matplotlib_rec(rec):
        return matplotlib.patches.Rectangle(
            (rec.x, rec.y),
            width=rec.width,height=rec.height,
            edgecolor='blue',
            linewidth=1.3,
            fill=False,
        )
    fig, ax = plt.subplots()
    
    for rec in recs:
        ax.add_patch(to_matplotlib_rec(rec))
        if show_ident:
            ax.text(rec.x+rec.width/2, rec.y+rec.height/2, rec.ident, fontsize=8)


    ax.set_xlim(left=-1, right=grid.width+1)
    ax.set_ylim(bottom=-1, top=grid.height)
    ax.set_title(f'Slicing layout', fontsize=8)
    
    # Hide the right and top spines
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)    
    
    plt.show()